![](img/563_banner.png)

# Lecture 8: Recommender Systems Part 2

UBC Master of Data Science program, 2021-22

Instructor: Varada Kolhatkar

## Lecture plan, imports, and LOs

### Imports 

In [1]:
import os
import random
import sys
import time

import numpy as np
import pandas as pd

sys.path.append("code/.")
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_validate, train_test_split

pd.set_option("display.max_colwidth", 0)

<br><br>

### Lecture plan

- Left-over iClicker questions from lecture 7 (~5 mins)
- Motivation (~5 mins)
- Content-based filtering (30 mins)
- Break (~5 mins)
- Questions for class discussion ( ~5 mins)
- Hybrid approaches sparse utility matrix (~s10 mins)
- Final comments, summary, reflection (~5 mins)
- Course evaluations (~5 mins)

<br><br>

### Learning outcomes

From this lecture, students are expected to be able to:
- Formulate the rating prediction problem as a supervised machine learning problem. 
- Create a content-based filter given ratings data and item features to predict missing ratings in the utility matrix.  
- Discuss differences, advantages and disadvantages between content-based filtering and collaborative filtering.
- Explain the idea of hybrid approaches at a high level. 
- Create and work with a sparse utility matrix for datasets with large number of items and users.   
- Name different kinds of data which occur in the context of recommendation systems. 
- Discuss important considerations in recommendation systems beyond error rate. 

## Recap and Motivation

### Recap: Recommender systems problem 

- We are usually given ratings data. 
- We use this data to create **utility matrix** which encodes interactions between users and items. 
- The utility matrix has many missing entries. 
- We defined recommendation systems problem as **matrix completion problem**. 

### Collaborative filtering approach  

- We use PCA-like approach to learn the latent features of users and items.
- The key idea is that the loss function only includes the available ratings. 
- So instead of using the regular PCA or `TruncatedSVD`, we used the `surprise` package, which only considers available ratings in the loss function. 

### Collaborative filtering

- "Unsupervised" learning 
    - We only have labels $Y$ (ratings $y_{ij}$ for user $i$ and item $j$) but no features.
- What if a new item shows up? 
    - You won't have any ratings information for that item.     

### Content-based filtering

- Supervised machine learning approach 
- Content-based filtering is suitable to predict ratings for new items.

<br><br><br><br>

## Content-based filtering

 

### Item and user features

- In collaborative filtering we assumed that we only have ratings data. 
- Usually there is some information available on items and users. 
- Examples
    - Netflix can describe movies as action, romance, comedy, documentaries. 
    - Amazon could describe books according to topics: math, languages, history. 
    - Tinder could describe people according to age, location, employment.
- Can we use this information to predict ratings in the utility matrix?   
    - Yes!

### Toy example: Movie recommendation

- Let's consider movie recommendation problem with the following toy data.

### Ratings data

In [2]:
toy_ratings = pd.read_csv("data/toy_ratings.csv")
toy_ratings

,user_id,movie_id,rating
0,Sam,Lion King,4
1,Sam,Jerry Maguire,4
2,Sam,Roman Holidays,5
3,Sam,Downfall,1
4,Eva,Titanic,2
5,Eva,Jerry Maguire,1
6,Eva,Inception,4
7,Eva,Man on Wire,5
8,Eva,The Social Dilemma,5
9,Pat,Titanic,3


In [3]:
N = len(np.unique(toy_ratings["user_id"]))
M = len(np.unique(toy_ratings["movie_id"]))
print("Number of users (N)                : %d" % N)
print("Number of movies (M)               : %d" % M)

Number of users (N)                : 4
Number of movies (M)               : 12


In [4]:
user_key = "user_id"
item_key = "movie_id"

In [5]:
user_mapper = dict(zip(np.unique(toy_ratings[user_key]), list(range(N))))
item_mapper = dict(zip(np.unique(toy_ratings[item_key]), list(range(M))))
user_inverse_mapper = dict(zip(list(range(N)), np.unique(toy_ratings[user_key])))
item_inverse_mapper = dict(zip(list(range(M)), np.unique(toy_ratings[item_key])))

### Utility matrix

Let's create a dense utility matrix for our toy dataset. 

In [6]:
def create_Y_from_ratings(data, N, M):
    Y = np.zeros((N, M))
    Y.fill(np.nan)
    for index, val in data.iterrows():
        n = user_mapper[val[user_key]]
        m = item_mapper[val[item_key]]
        Y[n, m] = val["rating"]

    return Y

### Utility matrix

In [7]:
Y = create_Y_from_ratings(toy_ratings, N, M)
utility_mat = pd.DataFrame(Y, columns=item_mapper.keys(), index=user_mapper.keys())
utility_mat

,A Beautiful Mind,Bambi,Cast Away,Downfall,Inception,Jerry Maguire,Lion King,Malcolm x,Man on Wire,Roman Holidays,The Social Dilemma,Titanic
Eva,NaN,NaN,NaN,NaN,4.0,1.0,NaN,NaN,5.0,NaN,5.0,2.0
Jim,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4.0,5.0,NaN,5.0,2.0
Pat,3.0,4.0,3.0,2.0,NaN,5.0,4.0,NaN,NaN,NaN,NaN,3.0
Sam,NaN,NaN,NaN,1.0,NaN,4.0,4.0,NaN,NaN,5.0,NaN,NaN


In [8]:
avg = np.nanmean(Y)

**Goal**: Predict missing entries in the utility matrix. 

In [9]:
import surprise
from surprise import SVD, Dataset, Reader, accuracy

Let's predict ratings with collaborative filtering.

In [10]:
reader = Reader()
data = Dataset.load_from_df(toy_ratings, reader)  # Load the data

trainset, validset = surprise.model_selection.train_test_split(
    data, test_size=0.01, random_state=42
)  # Split the data

In [11]:
k = 2
algo = SVD(n_factors=k, random_state=42)
algo.fit(trainset)
preds = algo.test(trainset.build_testset())

In [12]:
from collections import defaultdict

rating_preds = defaultdict(list)
for uid, iid, true_r, est, _ in preds:
    rating_preds[uid].append((iid, est))

In [13]:
rating_preds

defaultdict(list,
            {'Sam': [('Lion King', 3.5442874862516582),
              ('Jerry Maguire', 3.471958396420975),
              ('Downfall', 3.141157981632877),
              ('Roman Holidays', 3.6555436348053982)],
             'Jim': [('Lion King', 3.6494404051925047),
              ('The Social Dilemma', 3.8739407581035588),
              ('Titanic', 3.295718235231984),
              ('Man on Wire', 3.8839492577938532),
              ('Malcolm x', 3.6435176323135128)],
             'Pat': [('A Beautiful Mind', 3.4463313322323263),
              ('Bambi', 3.540418795140043),
              ('Jerry Maguire', 3.4582870107738803),
              ('Titanic', 3.1872411557123517),
              ('Cast Away', 3.4442142132704827),
              ('Lion King', 3.5286392016604875),
              ('Downfall', 3.133747883605952)],
             'Eva': [('The Social Dilemma', 3.6665140635371194),
              ('Jerry Maguire', 3.3423360343482957),
              ('Titanic', 3.113324069881

### Movie features

- Suppose we also have movie features. 

In [14]:
movie_feats_df = pd.read_csv("data/toy_movie_feats.csv", index_col=0)
movie_feats_df

,Action,Romance,Drama,Comedy,Children,Documentary
A Beautiful Mind,0,1,1,0,0,0
Bambi,0,0,1,0,1,0
Cast Away,0,1,1,0,0,0
Downfall,0,0,0,0,0,1
Inception,1,0,1,0,0,0
Jerry Maguire,0,1,1,1,0,0
Lion King,0,0,1,0,1,0
Malcolm x,0,0,0,0,0,1
Man on Wire,0,0,0,0,0,1
Roman Holidays,0,1,1,1,0,0


In [15]:
Z = movie_feats_df.to_numpy()
Z.shape

(12, 6)

- How can we use these features to predict missing ratings? 

### Overall idea

- Using the ratings data and movie features, we'll build **profiles for different users**. 
- Let's consider an example user **Pat**. 

### Pat's ratings

- We don't know anything about Pat but we know her ratings to movies. 

In [16]:
utility_mat.loc["Pat"]

A Beautiful Mind      3.0
Bambi                 4.0
Cast Away             3.0
Downfall              2.0
Inception            NaN 
Jerry Maguire         5.0
Lion King             4.0
Malcolm x            NaN 
Man on Wire          NaN 
Roman Holidays       NaN 
The Social Dilemma   NaN 
Titanic               3.0
Name: Pat, dtype: float64

- We also know about movies and their features. 
- If Pat gave a high rating to _Lion King_, it means that she liked the features of the movie. 

In [17]:
movie_feats_df.loc["Lion King"]

Action         0
Romance        0
Drama          1
Comedy         0
Children       1
Documentary    0
Name: Lion King, dtype: int64

### Supervised approach to rating prediction 

- We treat ratings prediction problem as a set of regression problems. 
- Given movie information, we create user profile for each user.
- Build regression model for each user and learn regression weights for each user. 

- We build a profile for users based on 
    - the movies they have watched
    - their rating for the movies
    - the features of the movies
- We train a personalized regression model for each user using this information. 


### Supervised approach to rating prediction 

For each user $i$ create a user profile as follows. 

- Consider all movies rated by $i$ and create `X` and `y` for the user: 
    - Each row in `X` contains the movie features of movie $j$ rated by $i$. 
    - Each value in `y` is the corresponding rating given to the movie $j$ by user $i$. 
- Fit a regression model using `X` and `y`. 
- Apply the model to predict ratings for new items! 

### Let's build user profiles 

- Build `X` and `y` for all users. 

In [18]:
from collections import defaultdict


def get_lr_data_per_user(ratings_df, d):
    lr_y = defaultdict(list)
    lr_X = defaultdict(list)
    lr_items = defaultdict(list)

    for index, val in ratings_df.iterrows():
        n = user_mapper[val[user_key]]
        m = item_mapper[val[item_key]]
        lr_X[n].append(Z[m])
        lr_y[n].append(val["rating"])
        lr_items[n].append(m)

    for n in lr_X:
        lr_X[n] = np.array(lr_X[n])
        lr_y[n] = np.array(lr_y[n])

    return lr_X, lr_y, lr_items

In [19]:
d = movie_feats_df.shape[1]
X_train_usr, y_train_usr, rated_items = get_lr_data_per_user(toy_ratings, d)

- What's going is be shape of each `X` and `y`?

### Examine user profiles 

- Let's examine some user profiles. 

In [20]:
def get_user_profile(user_name):
    X = X_train_usr[user_mapper[user_name]]
    y = y_train_usr[user_mapper[user_name]]
    items = rated_items[user_mapper[user_name]]
    movie_names = [item_inverse_mapper[item] for item in items]
    print("Profile for user: ", user_name)
    profile_df = pd.DataFrame(X, columns=movie_feats_df.columns, index=movie_names)
    profile_df["ratings"] = y
    return profile_df

### Pat's profile

In [21]:
get_user_profile("Pat")

Profile for user:  Pat


,Action,Romance,Drama,Comedy,Children,Documentary,ratings
Titanic,0,1,1,0,0,0,3
Lion King,0,0,1,0,1,0,4
Bambi,0,0,1,0,1,0,4
Cast Away,0,1,1,0,0,0,3
Jerry Maguire,0,1,1,1,0,0,5
Downfall,0,0,0,0,0,1,2
A Beautiful Mind,0,1,1,0,0,0,3


- Pat seems to like Children's movies and movies with Comedy. 
- Seems like she's not so much into romantic movies.  


### Eva's profile

In [22]:
get_user_profile("Eva")

Profile for user:  Eva


,Action,Romance,Drama,Comedy,Children,Documentary,ratings
Titanic,0,1,1,0,0,0,2
Jerry Maguire,0,1,1,1,0,0,1
Inception,1,0,1,0,0,0,4
Man on Wire,0,0,0,0,0,1,5
The Social Dilemma,0,0,0,0,0,1,5


- Eva hasn't rated many movies. There are not many rows. 
- Eva seems to like documentaries and action movies. 
- Seems like he's not so much into romantic movies.  

### Regression models for users

In [23]:
from sklearn.linear_model import Ridge


def train_for_usr(user_name, model=Ridge()):
    X = X_train_usr[user_mapper[user_name]]
    y = y_train_usr[user_mapper[user_name]]
    model.fit(X, y)
    return model


def predict_for_usr(model, movie_names):
    feat_vecs = movie_feats_df.loc[movie_names].values
    preds = model.predict(feat_vecs)
    return preds

### Regression model for Pat 

- What are the regression weights learned for Pat? 

In [24]:
user_name = "Pat"
pat_model = train_for_usr(user_name)
col = "Coefficients for %s" % user_name
pd.DataFrame(pat_model.coef_, index=movie_feats_df.columns, columns=[col])

,Coefficients for Pat
Action,0.000000
Romance,-0.020833
Drama,0.437500
Comedy,0.854167
Children,0.458333
Documentary,-0.437500


### Predictions for Pat
- How would Pat rate some movies she hasn't seen? 

In [25]:
movies_to_pred = ["Roman Holidays", "Malcolm x"]
pred_df = movie_feats_df.loc[movies_to_pred]
pred_df

,Action,Romance,Drama,Comedy,Children,Documentary
Roman Holidays,0,1,1,1,0,0
Malcolm x,0,0,0,0,0,1


In [26]:
user_name = "Pat"
preds = predict_for_usr(pat_model, movies_to_pred)
pred_df[user_name + "'s predicted ratings"] = preds
pred_df

,Action,Romance,Drama,Comedy,Children,Documentary,Pat's predicted ratings
Roman Holidays,0,1,1,1,0,0,4.145833
Malcolm x,0,0,0,0,0,1,2.437500


### Regression model for Eva 

- What are the regression weights learned for Eva? 

In [27]:
user_name = "Eva"
eva_model = train_for_usr(user_name)
col = "Coefficients for %s" % user_name
pd.DataFrame(eva_model.coef_, index=movie_feats_df.columns, columns=[col])

,Coefficients for Eva
Action,0.333333
Romance,-1.000000
Drama,-0.666667
Comedy,-0.666667
Children,0.000000
Documentary,0.666667


### Predictions for Eva

- What are the predicted ratings for Eva for a list of movies?

In [28]:
user_name = "Eva"
preds = predict_for_usr(eva_model, movies_to_pred)
pred_df[user_name + "'s predicted ratings"] = preds
pred_df

,Action,Romance,Drama,Comedy,Children,Documentary,Pat's predicted ratings,Eva's predicted ratings
Roman Holidays,0,1,1,1,0,0,4.145833,1.666667
Malcolm x,0,0,0,0,0,1,2.437500,4.666667


### Completing the utility matrix with content-based filtering

Here is the original utility matrix.  

In [29]:
utility_mat

,A Beautiful Mind,Bambi,Cast Away,Downfall,Inception,Jerry Maguire,Lion King,Malcolm x,Man on Wire,Roman Holidays,The Social Dilemma,Titanic
Eva,NaN,NaN,NaN,NaN,4.0,1.0,NaN,NaN,5.0,NaN,5.0,2.0
Jim,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4.0,5.0,NaN,5.0,2.0
Pat,3.0,4.0,3.0,2.0,NaN,5.0,4.0,NaN,NaN,NaN,NaN,3.0
Sam,NaN,NaN,NaN,1.0,NaN,4.0,4.0,NaN,NaN,5.0,NaN,NaN


- Using predictions per user, we can fill in missing entries in the utility matrix. 

In [30]:
from sklearn.linear_model import Ridge

models = dict()
pred_lin_reg = np.zeros((N, M))

for n in range(N):
    models[n] = Ridge()
    models[n].fit(X_train_usr[n], y_train_usr[n])
    pred_lin_reg[n] = models[n].predict(Z)

In [31]:
pd.DataFrame(pred_lin_reg, columns=item_mapper.keys(), index=user_mapper.keys())

,A Beautiful Mind,Bambi,Cast Away,Downfall,Inception,Jerry Maguire,Lion King,Malcolm x,Man on Wire,Roman Holidays,The Social Dilemma,Titanic
Eva,2.333333,3.333333,2.333333,4.666667,3.666667,1.666667,3.333333,4.666667,4.666667,1.666667,4.666667,2.333333
Jim,2.575000,3.075000,2.575000,4.450000,3.150000,2.575000,3.075000,4.450000,4.450000,2.575000,4.450000,2.575000
Pat,3.291667,3.770833,3.291667,2.437500,3.312500,4.145833,3.770833,2.437500,2.437500,4.145833,2.437500,3.291667
Sam,3.810811,3.675676,3.810811,1.783784,3.351351,4.270270,3.675676,1.783784,1.783784,4.270270,1.783784,3.810811


### Content-based vs. collaborative filtering 

- Latent-factor approach to collaborative filtering, where we reconstruct rating for user $i$ and item $j$ as: 
$$\hat{y}_{ij} = w_j^T z_{i}$$

    - $w_j^T$ are "hidden" features of item $j$
    - $z_i$ are "hidden" features of user $i$


- A linear model approach to content-based filtering, where we reconstruct rating for user $i$ and item $j$ as:  
$$\hat{y}_{ij} = w^T x_{ij}$$
    - $x_{ij}$ is a feature vector for user $i$ and item $j$
    - $w$ are the feature vectors for user $i$
    - Our usual supervised learning setup for linear regression.  



### More comments on content-based filtering

- The feature matrix for movies can contain different types of features.
    - Example: Plot of the movie (text features), actors (categorical features), year of the movie, budget and revenue of the movie (numerical features). 
    - You'll apply our usual preprocessing techniques to these features. 
- If you have enough data, you could also carry out hyperparameter tuning with cross-validation for each model.
- Finally, although we have been talking about linear models above, you can use any regression model of your choice. 

### Advantages of content-based filtering 

- We don't need many users to provide ratings for an item. 
- Each user is modeled separately, so you might be able to capture uniqueness of taste. 
- Since you can obtain the features of the items, you can immediately recommend new items. 
    - This would not have been possible with collaborative filtering. 
- Recommendations are interpretable.
    - You can explain to the user why you are recommending an item because you have learned weights. 
    

### Disadvantages of content-based filtering 

- Feature acquisition and feature engineering
    - What features should we use to explain the difference in ratings? 
    - Obtaining those features for each item might be very expensive. 
- Less diversity: hardly recommend an item outside the user's profile. 
- Cold start: When a new user shows up, you don't have any information about them.

<br><br><br><br>

## ❓❓ Questions for you
iClicker cloud join link: https://join.iclicker.com/MA16T

### Exercise 8.1 Select all of the following statements which are **True** (iClicker)

- (A) In content-based filtering we leverage available item features in addition to similarity between users.
- (B) In content-based filtering you represent each user in terms of **known** features of items whereas in collaborative filtering each user is represented with **latent** features of items. 
- (C) In the set up of content-based filtering we discussed, if you have a new movie, you would have problems predicting ratings for that movie. 
- (D) Interpretation of recommendations might be easier with content-based filtering compared to collaborative filtering. 
- (E) In content-based filtering if a user has a number of ratings in the training utility matrix but does not have any ratings in the validation utility matrix then we won't be able to calculate RMSE for the validation set. 
<br><br><br><br>

```{admonition} Exercise 8.1: V's Solutions!
:class: tip, dropdown

- (A) False
- (B) True
- (C) False
- (D) True 
- (E) False
```

### Discussion questions
- Discuss similarities and differences between the Word2Vec style recommender we saw last week and the recommender systems we learned this week. 

<br><br><br><br>

## Hybrid approaches
<hr>

### General idea 
- Both collaborative filtering and content-based filtering suffer from shortcomings. 
- Collaborative filtering does not predict well for new movies/users.
    - New movies don't yet have ratings, and new users haven't rated anything.
- Content-based approaches are less diverse. 
- We are not exploiting information about similarity between users. 
- Most of the recommenders used in practice are hybrid recommenders which combine the best of the two worlds.  

### Hybrid approach 

Hybrid approaches combine content-based and collaborative filtering: [SVDfeature](https://www.jmlr.org/papers/v13/chen12a.html) (won "KDD Cup" in 2011 and 2012).

$$\hat{y}_{ij} = \beta + \beta_i + \beta_j + w^TX_{ij} + w_j^Tz_i$$ 

- $\beta$ is average rating across all users/movies
- $\beta_i$ is average rating for user $i$
- $\beta_j$ average rating for movie $j$
- $w^TX_{ij}$ is our linear model based on user/movie features $x_{ij}$ 
- $w_j^Tz_i$ factors learned by our collaboative filtering model.  
- Note that $x_{ij}$ is a feature vector for user $i$ and movie $j$. 
- Also, $w$ and $w_j$ are different parameters. 
- Supervised learning can predict for new movies! 


In my quick search, I didn't find an easy-to-use Python package for this. 

<br><br><br><br>

## Sparse utility matrix

- Recommender systems work best when there is a large amount of data. 
- So far we've been working with small datasets. 

Let's using the [Amazon product data set](http://jmcauley.ucsd.edu/data/amazon/). The authors of the data set have asked for the following citations:

> Ups and downs: Modeling the visual evolution of fashion trends with one-class collaborative filtering.
> R. He, J. McAuley.
> WWW, 2016.
> 
> Image-based recommendations on styles and substitutes.
> J. McAuley, C. Targett, J. Shi, A. van den Hengel.
> SIGIR, 2015.

We will focus on the Patio, Lawn, and Garden section. You can download the [ratings here](http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Patio_Lawn_and_Garden.csv). 

Let's load the data. 

In [32]:
filename = "ratings_Patio_Lawn_and_Garden.csv"

with open(os.path.join("data", filename), "rb") as f:
    ratings = pd.read_csv(f, names=("user", "item", "rating", "timestamp"))
ratings.head()

,user,item,rating,timestamp
0,A2VNYWOPJ13AFP,0981850006,5.0,1259798400
1,A20DWVV8HML3AW,0981850006,5.0,1371081600
2,A3RVP3YBYYOPRH,0981850006,5.0,1257984000
3,A28XY55TP3Q90O,0981850006,5.0,1314144000
4,A3VZW1BGUQO0V3,0981850006,5.0,1308268800


In [33]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 993490 entries, 0 to 993489
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user       993490 non-null  object 
 1   item       993490 non-null  object 
 2   rating     993490 non-null  float64
 3   timestamp  993490 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 30.3+ MB


We'd like to construct the utility matrix `Y`. However, let's see how big it would be:

In [34]:
def get_stats(ratings, item_key="item", user_key="user"):
    print("Number of ratings:", len(ratings))
    print("The average rating:", np.mean(ratings["rating"]))
    N = len(set(ratings[user_key]))
    M = len(set(ratings[item_key]))
    print("Number of users:", N)
    print("Number of items:", M)
    print("Fraction nonzero:", len(ratings) / (N * M))
    print("Size of full Y matrix (GB):", (N * M) * 8 / 1e9)
    return N, M


N, M = get_stats(ratings)

Number of ratings: 993490
The average rating: 4.006400668350965
Number of users: 714791
Number of items: 105984
Fraction nonzero: 1.3114269915944552e-05
Size of full Y matrix (GB): 606.051274752


606 GB! That is way too big. We don't want to create that matrix! On the other hand, we see that we only have about 1 million ratings, which would be 8 MB or so ($10^6$ numbers $\times$ 8 bytes per number). Much more manageable!

Let's create a sparse representation of our utility matrix $Y$. 

In [35]:
from scipy.sparse import csr_matrix as sparse_matrix

user_key = "user"
item_key = "item"
user_mapper = dict(zip(np.unique(ratings[user_key]), list(range(N))))
item_mapper = dict(zip(np.unique(ratings[item_key]), list(range(M))))

user_inverse_mapper = dict(zip(list(range(N)), np.unique(ratings[user_key])))
item_inverse_mapper = dict(zip(list(range(M)), np.unique(ratings[item_key])))

In [36]:
def create_Y(ratings, N, M, user_key="user", item_key="item"):
    """
    Creates a sparse matrix using scipy.csr_matrix and mappers to relate indexes to items' id.

    Parameters:
    -----------
    ratings: pd.DataFrame
        the ratings to be stored in the matrix;
    N: int
        the number of users
    M: int
        the number of items
    user_key: string
        the column in ratings that contains the users id
    item_key: string
        the column in ratings that contains the items id

    Returns:
    --------
    Y: np.sparse
        the sparse matrix containing the ratings.
    """
    user_ind = [user_mapper[i] for i in ratings[user_key]]
    item_ind = [item_mapper[i] for i in ratings[item_key]]
    Y = sparse_matrix((ratings["rating"], (user_ind, item_ind)), shape=(N, M))
    return Y

In [37]:
Y = create_Y(ratings, N, M)
Y

<714791x105984 sparse matrix of type '<class 'numpy.float64'>'
	with 993490 stored elements in Compressed Sparse Row format>

Note the shape of `Y`: our rows are the users, and the columns are products.

In [38]:
# sanity check
print(Y.shape)  # should be number of items by number of users
print(Y.nnz)  # number of nonzero elements -- should equal number of ratings
print(f"Using sparse matrix data structure, the size of X is: {Y.data.nbytes/1e6}mb")

(714791, 105984)
993490
Using sparse matrix data structure, the size of X is: 7.94792mb


### Let's try `surprise` package on this

In [39]:
ratings = ratings.drop(columns=["timestamp"])

In [40]:
import surprise
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate

reader = Reader()
data = Dataset.load_from_df(ratings, reader)  # Load the data
k = 10
algo = SVD(n_factors=k, random_state=42)
pd.DataFrame(cross_validate(algo, data, measures=["RMSE", "MAE"], cv=5, verbose=True))

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2899  1.2915  1.2896  1.2915  1.2911  1.2907  0.0008  
MAE (testset)     1.0197  1.0206  1.0187  1.0211  1.0204  1.0201  0.0008  
Fit time          51.16   43.29   33.87   34.28   31.65   38.85   7.33    
Test time         6.45    2.64    2.46    2.29    2.64    3.30    1.58    


,test_rmse,test_mae,fit_time,test_time
0,1.289854,1.019705,51.160298,6.446292
1,1.291460,1.020583,43.291631,2.636477
2,1.289556,1.018744,33.871170,2.463427
3,1.291472,1.021107,34.279560,2.293967
4,1.291086,1.020441,31.649730,2.635332


### Displaying product urls 

Let's consider the following product. 
- [Mr Grill - 18" Luxury Oak Barbecue Spatula / Turner](https://www.amazon.com/dp/B00IJB5MCS). 


In [41]:
from IPython.core.display import HTML, display

url_amazon = "https://www.amazon.com/dp/%s"


def disp_url(item_id):
    url = url_amazon % item_id
    display(HTML('<a href="%s">%s</a>' % (url, url)))

/var/folders/ky/533nd9l512l5cmmk_kzmzj9m0000gp/T/ipykernel_6281/4119128594.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display


In [42]:
grill_spatula = "B00IJB5MCS"
grill_spatula_ind = item_mapper[grill_spatula]
grill_spatula_vec = Y[grill_spatula_ind]
disp_url(grill_spatula)

<br><br><br><br>

### Final comments and summary

### What did we cover? 

- There is a big world of recommendation systems out there. We talked about some basic traditional approaches to recommender systems. 
    - collaborative filtering 
    - content-based filtering 
    - hybrid approaches 

If you want to know more advanced approaches to recommender systems, watch this 4-hour summer school tutorial by Xavier Amatriain, Research/Engineering Director @ Netflix.  

- [Part1](https://www.youtube.com/watch?v=bLhq63ygoU8)
- [Part2](https://www.youtube.com/watch?v=mRToFXlNBpQ)


### Beyond error rate in recommendation systems 

- If a system gives the best RMSE it doesn't necessarily mean that it's going to give best recommendations. 
- In recommendation systems we do not have ground truth.
- Just training your model and evaluating it offline is not ideal. 
- Other aspects such as simplicity, interpretation, code maintainability are equally (if not more) important than best validation error. 
- Winning system of Netflix Challenge was never adopted.
    - Big mess of ensembles was not really maintainable 
- There are other considerations. 

### Other issues important in recommender systems

### Are these good recommendations? 

You are looking for water shoes and at the moment you are looking at [VIFUUR Water Sports Shoes](https://www.amazon.ca/VIFUUR-Barefoot-Quick-Dry-Blue-38-39/dp/B0753DL15Y), are these good recommendations? 

![](img/reco-diversity.png)


Now suppose you've recently bought VIFUUR Water Sports Shoes and rated them highly. Are these good recommendations now? 
- Not really. Even though you really liked them you don't need them anymore. You want some non-Water Sports Shoes recommendations.
- **Diversity** is about how different are the recommendations. 
    - Another example: Even if you really really like Star Wars, you might want non-Star-Wars suggestions.    
- But be careful. We need a balance here. 

### Are these good recommendations? 

![](img/freshness.png)

- Some of these books don't have many ratings but it might be a good idea to recommend "fresh" things. 
- **Freshness**: people tend to get more excited about new/surprising things.    

- But again you need a balance here. What would happen if you keep surprising the user all the time? 
- There might be **trust** issues. 
- Another aspect of trust is explaining your recommendation, i.e., telling the user why you made a recommendation. This gives the user an opportunity to understand why your recommendations could be interesting to them.   
    
![](img/recommendation-explanation.png)    

[Source](https://sudonull.com/post/65548-10-lessons-of-the-Quora-recommendation-system-Retail-Rocket-Blog)

**Persistence**: how long should recommendations last?
- If you keep not clicking on a recommendation, should it remain a recommendation?

**Social recommendation**: what did your friends watch?
- Many recommenders	are	now	connected to social	networks.
- "Login using you Facebook	account".
- Often, people	like similar movies	to their friends.
- If we get a new user, then recommendations are based on friend's preferences. 

### Types of data 

- Explicit data: ratings, thumbs up, etc. 
- Implicit data: collected from the users' behaviour (e.g., mouse clicks, purchases, time spent doing something)
- Trust implicit data that costs something, like time or even money. 
    - this makes it harder to fraud

### Reminder

- Recommendation systems can have terrible consequences. 
- Ask hard and uncomfortable questions to yourself (and to your employer if possible) before implementing and deploying a recommendation system.  

<br><br><br><br>

## ❓❓ Questions for you
iClicker cloud join link: https://join.iclicker.com/MA16T

### Exercise 8.2 Select all of the following statements which are **True** (iClicker)
- (A) If you have a large number of users and items, it would be a problem to evaluate the reconstructed matrix against train and validation sets.
- (B) `SVDFeature` hybrid model creates separate rating prediction models with content-based and collaborative filtering.  
- (C) Suppose a user views 1 minute of a 10 minute YouTube video. It's an explicit input.  

```{admonition} Exercise 8.2: V's Solutions!
:class: tip, dropdown

- (A) False
- (B) False
- (C) False
```

### Discussion questions

- Discuss memory-related problems that may cause when dealing with large number of users and items. 
- We have been ignoring the timestamp column in ratings datasets. How you might use this information when making recommendations? 

### (Optional) Practice exercises for you

Use scikit-learn's [NearestNeighbors](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html) object (which uses Euclidean distance by default) to find the 10 most similar items to [Mr Grill - 18" Luxury Oak Barbecue Spatula / Turner](https://www.amazon.com/dp/B00IJB5MCS) using Euclidean distance and cosine distance. Which distance metric is giving you better recommendations? 

> Try it out on your own or with your friends. I might not get a chance to post solutions for these questions. 

<br><br><br><br>

## Course roadmap

- Week 1 ✅
    - Clustering    
- Week 2 ✅
    - Dimensionality reduction
- Week 3 ✅
    - Word embeddings
- Week 4 ✅
    - Recommendation systems              

![](img/eva-accomplished.png)

### Time for course evaluations

That's all for this course! It was fun teaching you this material. Thanks for your support, feedback, and great questions ❤️!

I would love to hear your thought on this course. When you get a chance, it'll be great if you fill in the evaluation survey for this course on [Canvas](https://canvas.ubc.ca/courses/83554/external_tools/4732). 

The evaluation closing date is: **March 25, 2022**

<br><br><br><br>

## Resources

- If you want to know more advanced approaches to recommender systems, watch this 4-hour summer school tutorial by Xavier Amatriain, Research/Engineering Director @ Netflix. ([Part1](https://www.youtube.com/watch?v=bLhq63ygoU8), [Part2](https://www.youtube.com/watch?v=mRToFXlNBpQ))

- [10 lessons of the Quora recommendation system](https://sudonull.com/post/65548-10-lessons-of-the-Quora-recommendation-system-Retail-Rocket-Blog)

<br><br><br><br>